In [32]:
#單層卷積受限玻爾茲曼機模型

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import time

# 定義 ConvRBM 模型
class ConvRBM(nn.Module):
    def __init__(self, num_visible, num_hidden, kernel_size):
        super(ConvRBM, self).__init__()
        self.conv = nn.Conv2d(num_visible, num_hidden, kernel_size, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.unpool = nn.Upsample(scale_factor=2, mode='nearest')
        self.deconv = nn.ConvTranspose2d(num_hidden, num_visible, kernel_size, stride=1, padding=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        h = self.conv(x)
        h = self.sigmoid(h)
        h_pooled = self.pool(h)
        h_unpooled = self.unpool(h_pooled)
        x_reconstructed = self.deconv(h_unpooled)
        x_reconstructed = self.sigmoid(x_reconstructed)
        return x_reconstructed, h

# 超參數設定
num_visible = 1  # 輸入圖像通道數（灰階圖像為1）
num_hidden = 32  # 隱藏層特徵圖的通道數
kernel_size = 3  # 卷積核的大小

# 載入訓練資料
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# 初始化 ConvRBM 模型
model = ConvRBM(num_visible, num_hidden, kernel_size)

# 定義損失函數和優化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 訓練 ConvRBM 模型
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 訓練時間計算
start_time = time.time()

for epoch in range(num_epochs):
    total_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        reconstructed_data, _ = model(data)
        loss = criterion(reconstructed_data, data)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, total_loss))

end_time = time.time()
training_time = end_time - start_time
print('Total training time: {:.2f} seconds'.format(training_time))

# 測試集 RMSE 和 PSNR
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True)

model.eval()
with torch.no_grad():
    mse_loss = 0
    psnr = 0
    num_samples = 0
    for data, _ in test_loader:
        data = data.to(device)
        reconstructed_data, _ = model(data)
        mse_loss += criterion(reconstructed_data, data).item()
        psnr += 10 * torch.log10(1. / criterion(reconstructed_data, data)).item()
        num_samples += 1

    mse_loss /= num_samples
    psnr /= num_samples

    print('Test RMSE: {:.4f}'.format(np.sqrt(mse_loss)))
    print('Test PSNR: {:.4f}'.format(psnr))




Epoch [1/10], Loss: 17.5174
Epoch [2/10], Loss: 3.2639
Epoch [3/10], Loss: 1.9642
Epoch [4/10], Loss: 1.5923
Epoch [5/10], Loss: 1.3670
Epoch [6/10], Loss: 1.1758
Epoch [7/10], Loss: 1.0175
Epoch [8/10], Loss: 0.8843
Epoch [9/10], Loss: 0.7874
Epoch [10/10], Loss: 0.7298
Total training time: 397.55 seconds
Test RMSE: 0.0274
Test PSNR: 31.6279


In [33]:
# 卷积神经网络模型

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import time
import numpy as np

# 定义卷积神经网络模型
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 超参数设置
num_epochs = 10
batch_size = 64
learning_rate = 0.001

# 载入训练数据
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 初始化卷积神经网络模型
model = ConvNet()

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 转移到GPU上进行训练（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 训练时间计算
start_time = time.time()

for epoch in range(num_epochs):
    total_loss = 0
    for batch_idx, (data, labels) in enumerate(train_loader):
        data = data.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, total_loss))

end_time = time.time()
training_time = end_time - start_time
print('Total training time: {:.2f} seconds'.format(training_time))

# 测试集准确率
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for data, labels in test_loader:
        data = data.to(device)
        labels = labels.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print('Test Accuracy: {:.2f}%'.format(accuracy))

Epoch [1/10], Loss: 160.8797
Epoch [2/10], Loss: 44.9903
Epoch [3/10], Loss: 31.0271
Epoch [4/10], Loss: 23.0282
Epoch [5/10], Loss: 17.9684
Epoch [6/10], Loss: 12.9549
Epoch [7/10], Loss: 11.0559
Epoch [8/10], Loss: 8.2944
Epoch [9/10], Loss: 6.5463
Epoch [10/10], Loss: 6.4619
Total training time: 443.72 seconds
Test Accuracy: 99.12%


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import time

# 定義卷積神經網路模型
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(7 * 7 * 64, 10)

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.pool1(out)
        out = self.conv2(out)
        out = self.relu2(out)
        out = self.pool2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# 超參數設定
num_epochs = 10
batch_size = 64
learning_rate = 0.001

# 載入訓練資料
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 初始化卷積神經網路模型
model = ConvNet()

# 定義損失函數和優化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 訓練卷積神經網路模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 訓練時間計算
start_time = time.time()

total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # 前向傳播
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 反向傳播和優化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

end_time = time.time()
training_time = end_time - start_time
print('Total training time: {:.2f} seconds'.format(training_time))

# 測試集準確率
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print('Test Accuracy: {:.2f}%'.format(accuracy))

Epoch [1/10], Step [100/938], Loss: 0.3242
Epoch [1/10], Step [200/938], Loss: 0.3169
Epoch [1/10], Step [300/938], Loss: 0.2012
Epoch [1/10], Step [400/938], Loss: 0.1394
Epoch [1/10], Step [500/938], Loss: 0.1725
Epoch [1/10], Step [600/938], Loss: 0.0599
Epoch [1/10], Step [700/938], Loss: 0.1911
Epoch [1/10], Step [800/938], Loss: 0.2970
Epoch [1/10], Step [900/938], Loss: 0.0143
Epoch [2/10], Step [100/938], Loss: 0.0752
Epoch [2/10], Step [200/938], Loss: 0.0622
Epoch [2/10], Step [300/938], Loss: 0.0240
Epoch [2/10], Step [400/938], Loss: 0.0325
Epoch [2/10], Step [500/938], Loss: 0.0239
Epoch [2/10], Step [600/938], Loss: 0.0250
Epoch [2/10], Step [700/938], Loss: 0.0474
Epoch [2/10], Step [800/938], Loss: 0.0139
Epoch [2/10], Step [900/938], Loss: 0.0199
Epoch [3/10], Step [100/938], Loss: 0.0159
Epoch [3/10], Step [200/938], Loss: 0.0036
Epoch [3/10], Step [300/938], Loss: 0.0324
Epoch [3/10], Step [400/938], Loss: 0.0349
Epoch [3/10], Step [500/938], Loss: 0.1048
Epoch [3/10

In [29]:
# 稀疏編碼模型

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import time

# 定義稀疏編碼模型
class SparseCoding(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SparseCoding, self).__init__()
        self.encoder = nn.Linear(input_size, hidden_size)
        self.decoder = nn.Linear(hidden_size, input_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        h = self.relu(self.encoder(x))
        x_reconstructed = self.relu(self.decoder(h))
        return x_reconstructed, h

# 超參數設定
input_size = 784  # MNIST 圖像尺寸為 28x28，展開為 784 的向量
hidden_size = 32  # 隱藏層特徵數量

# 載入訓練資料
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# 初始化稀疏編碼模型
model = SparseCoding(input_size, hidden_size)

# 定義損失函數和優化器
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 訓練稀疏編碼模型
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 訓練時間計算
start_time = time.time()

for epoch in range(num_epochs):
    total_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.view(data.size(0), -1).to(device)
        optimizer.zero_grad()
        reconstructed_data, _ = model(data)
        loss = criterion(reconstructed_data, data)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, total_loss))

end_time = time.time()
training_time = end_time - start_time
print('Total training time: {:.2f} seconds'.format(training_time))

# 測試集重建誤差
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True)

model.eval()
with torch.no_grad():
    reconstruction_error = 0
    num_samples = 0
    for data, _ in test_loader:
        data = data.view(data.size(0), -1).to(device)
        reconstructed_data, _ = model(data)
        reconstruction_error += criterion(reconstructed_data, data).item()
        num_samples += 1

    reconstruction_error /= num_samples

    print('Test reconstruction error: {:.4f}'.format(reconstruction_error))

Epoch [1/10], Loss: 888.8231
Epoch [2/10], Loss: 886.8463
Epoch [3/10], Loss: 886.8464
Epoch [4/10], Loss: 886.8461
Epoch [5/10], Loss: 886.8453
Epoch [6/10], Loss: 886.8448
Epoch [7/10], Loss: 886.8443
Epoch [8/10], Loss: 886.8464
Epoch [9/10], Loss: 886.8443
Epoch [10/10], Loss: 886.8463
Total training time: 169.24 seconds
Test reconstruction error: 0.9457


In [41]:
import numpy as np
import cv2
import time
from skimage.metrics import peak_signal_noise_ratio, mean_squared_error
from skimage.transform import resize

# 生成训练图像
width = 256
height = 256

# 生成随机灰度图像作为训练图像
training_image = np.random.randint(0, 256, (height, width), dtype=np.uint8)

# 设置超分辨率倍数
scale_factor = 2

# 使用Bicubic模型进行超解析度处理
start_time = time.time()
bicubic_image = cv2.resize(training_image, (width*scale_factor, height*scale_factor), interpolation=cv2.INTER_CUBIC)
bicubic_time = time.time() - start_time
bicubic_rmse = np.sqrt(mean_squared_error(training_image, bicubic_image))

if bicubic_rmse == 0:
    bicubic_psnr = float('inf')
else:
    bicubic_psnr = peak_signal_noise_ratio(training_image, bicubic_image)

# 使用卷积受限玻尔兹曼机模型进行超解析度处理
# 需要使用相应的库进行模型训练和图像重建
# 这里只是一个示例，具体实现可能会有所不同

# 使用卷积神经网络模型进行超解析度处理
# 需要使用相应的库进行模型训练和图像重建
# 这里只是一个示例，具体实现可能会有所不同

# 使用稀疏编码模型进行超解析度处理
start_time = time.time()
# 实现稀疏编码模型，例如使用Lasso算法
# 这里只是一个示例，具体实现可能会有所不同
lasso_image = training_image  # 使用训练图像作为稀疏编码模型的结果
sparse_coding_time = time.time() - start_time
sparse_coding_rmse = np.sqrt(mean_squared_error(training_image, lasso_image))

if sparse_coding_rmse == 0:
    sparse_coding_psnr = float('inf')
else:
    sparse_coding_psnr = peak_signal_noise_ratio(training_image, lasso_image)

# 打印结果
print("Bicubic模型:")
print("训练时间: %.2f 秒" % bicubic_time)
print("RMSE值: %.2f" % bicubic_rmse)
print("PSNR值: %.2f" % bicubic_psnr)
print()

print("卷积受限玻尔兹曼机模型:")
# 打印训练时间、RMSE值和PSNR值
print()

print("卷积神经网络模型:")
# 打印训练时间、RMSE值和PSNR值
print()

print("稀疏编码模型:")
print("训练时间: %.2f 秒" % sparse_coding_time)
print("RMSE值: %.2f" % sparse_coding_rmse)
print("PSNR值: %.2f" % sparse_coding_psnr)

ValueError: Input images must have the same dimensions.

In [35]:
pip install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 MB 4.0 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
